In [1]:
from BERT_model import *
import BERT_model
import numpy as np
import torch
from torch import nn

In [2]:
import pandas as pd

df = pd.read_csv('../data/csv/MBTI_min_1000.csv')

In [3]:
df = df.groupby(['type'])
func = lambda g: g.sort_values(by = 'type', ascending=False)[:1]

df = df.apply(func)

In [4]:
df_train, df_val, df_test = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

12 2 2


In [9]:
EPOCHS = 2
model = BertClassifier()
LR = 1e-6

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    device, criterion, optimizer = BERT_model.Get_Device_Optimizer(model, learning_rate)
    
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                output = BERT_model.GetModelOutput(model, train_input, train_label, device)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                total_acc_train = BERT_model.Get_Total_Accuracy(output, train_label)                

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    output = BERT_model.GetModelOutput(model, val_input, val_label, device)                   

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    total_acc_val = BERT_model.Get_Total_Accuracy(output, val_label)
                    
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
              
train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 6/6 [02:38<00:00, 26.48s/it]


Epochs: 1 | Train Loss:  1.397                 | Train Accuracy:  0.083                 | Val Loss:  1.327                 | Val Accuracy:  0.000


100%|██████████| 6/6 [02:16<00:00, 22.70s/it]


Epochs: 2 | Train Loss:  1.503                 | Train Accuracy:  0.000                 | Val Loss:  1.551                 | Val Accuracy:  0.000


In [7]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    device = BERT_model.Get_Device(model)

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            output = BERT_model.GetModelOutput(model, test_input, test_label, device)
            total_acc_test = BERT_model.Get_Total_Accuracy(output, test_label)            
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.000
